In [1]:
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm
import math
import missingno as msno
import dateparser
import time
from numpy.core.numeric import NaN

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn import linear_model
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import matplotlib as mpl
from wordcloud import WordCloud
from sklearn.metrics import accuracy_score, log_loss
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC, LinearSVC
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn import datasets, svm
from sklearn.model_selection import GridSearchCV

import xgboost as xgb

In [2]:
test = pd.read_csv("../../test_final.csv")
train = pd.read_csv("../../train_final.csv")

train = train.drop(columns=['Unnamed: 0'])
test = test.drop(columns='Unnamed: 0')

booking_bool = train['booking_bool'].to_numpy()
test_ids = test['srch_id'].to_numpy()

In [10]:
train

,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,...,comp_inv,comp_percent,month,daypart,prop_location_score2,%booked_prop,%clicked_prop,relative_price_usd,booked_prop,clicked_prop
0,1,12,187,0.0,0.0,219,893,3,3.5,1,...,0.000000,0.0,4,2,0.04,1.633987,2.614379,104.77,1.633987,2.614379
1,1,12,187,0.0,0.0,219,10404,4,4.0,1,...,0.333333,0.0,4,2,0.01,1.543739,2.572899,170.74,1.543739,2.572899
2,1,12,187,0.0,0.0,219,21315,3,4.5,1,...,0.000000,0.0,4,2,0.02,0.362976,0.907441,179.80,0.362976,0.907441
3,1,12,187,0.0,0.0,219,27348,2,4.0,1,...,0.250000,5.0,4,2,0.01,1.304348,2.391304,602.77,1.304348,2.391304
4,1,12,187,0.0,0.0,219,29604,4,3.5,1,...,0.000000,0.0,4,2,0.10,3.609023,5.112782,143.58,3.609023,5.112782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2989384,332784,24,219,0.0,0.0,100,114177,4,4.5,1,...,0.000000,19.0,6,3,0.09,3.612167,5.513308,300.74,3.612167,5.513308
2989385,332784,24,219,0.0,0.0,100,115562,4,4.0,1,...,0.000000,0.0,6,3,0.07,2.491103,3.914591,213.15,2.491103,3.914591
2989386,332784,24,219,0.0,0.0,100,121275,4,4.0,0,...,0.000000,0.0,6,3,0.04,2.955665,3.448276,193.87,2.955665,3.448276
2989387,332784,24,219,0.0,0.0,100,128677,3,3.5,1,...,0.000000,18.0,6,3,0.10,3.693931,6.332454,107.05,3.693931,6.332454


In [9]:
test

,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,...,comp_inv,comp_percent,month,daypart,prop_location_score2,%booked_prop,%clicked_prop,relative_price_usd,booked_prop,clicked_prop
0,1,24,216,0.0,0.0,219,3180,3,4.5,1,...,0.0,0.000000,2,3,0.07,4.301075,4.301075,119.00,4.301075,4.301075
1,1,24,216,0.0,0.0,219,5543,3,4.5,1,...,0.0,0.000000,2,3,0.08,5.327869,6.147541,118.00,5.327869,6.147541
2,1,24,216,0.0,0.0,219,14142,2,3.5,1,...,0.0,10.000000,2,3,0.06,1.342282,2.684564,49.00,1.342282,2.684564
3,1,24,216,0.0,0.0,219,22393,3,4.5,1,...,0.0,0.000000,2,3,0.06,1.379310,1.379310,143.00,1.379310,1.379310
4,1,24,216,0.0,0.0,219,24194,3,4.5,1,...,0.0,0.000000,2,3,0.20,1.463415,1.951220,79.00,1.463415,1.951220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4959178,332787,24,216,0.0,0.0,117,32019,4,3.5,0,...,0.0,49.500000,5,2,0.06,0.000000,0.000000,66.07,0.000000,0.000000
4959179,332787,24,216,0.0,0.0,117,33959,4,3.0,1,...,0.0,19.000000,5,2,0.30,10.000000,10.000000,67.10,10.000000,10.000000
4959180,332787,24,216,0.0,0.0,117,35240,4,0.0,0,...,0.0,24.666667,5,2,0.00,0.000000,0.000000,73.91,0.000000,0.000000
4959181,332787,24,216,0.0,0.0,117,94437,4,0.0,0,...,0.0,27.500000,5,2,0.09,0.000000,16.666667,66.07,0.000000,16.666667


In [8]:
train = train.drop(columns=['Unnamed: 0.1'])
test = test.drop(columns='Unnamed: 0.1')

In [ ]:
# maybe a better score function, we could try this instead of booking bool and see what it does
# score = []

# for i in range(len(train[train.columns[1]])):
#     if train['booking_bool'][i] == 1:
#         score.append(5)
#     elif train['click_bool'][i] == 1:
#         score.append(1)
#     else:
#         score.append(0)

# train['score'] = score

In [5]:
click_bool = train['click_bool']
train = train.drop(columns=['click_bool'])

In [6]:
for column in train.columns:
    if column not in test.columns:
        train = train.drop(columns=column)

print(len(test.columns))
print(len(train.columns))

36
36


In [7]:
len(train.index)
# check hier ff of length goed is!

2989389

In [12]:
print("nu start regressor")

# pas dit aan naar random forest!! ik weet niet iets van de parameters ervoor.

clf = RandomForestClassifier().fit(train, booking_bool)

# boost = xgb.XGBRegressor().fit(train, booking_bool)

print("fitten klaar nu predicten")

# boost_predictions = boost.predict(test)
clf_predictions = clf.predict_proba(test)

nu start regressor
fitten klaar nu predicten


In [15]:
test_results = pd.DataFrame()
test_results['srch_id'] = test_ids
test_results['prop_id'] = [int(x) for x in test['prop_id']]

print("predicten klaar nu appenden en dan omzetten sorten en exporten")



predicten klaar nu appenden en dan omzetten sorten en exporten


In [16]:
test_results

,srch_id,prop_id
0,1,3180
1,1,5543
2,1,14142
3,1,22393
4,1,24194
...,...,...
4959178,332787,32019
4959179,332787,33959
4959180,332787,35240
4959181,332787,94437


In [17]:
clf_predictions

array([[0.95, 0.05],
       [0.91, 0.09],
       [0.95, 0.05],
       ...,
       [1.  , 0.  ],
       [0.98, 0.02],
       [0.88, 0.12]])

In [21]:
pred_1 = []
# TODO: pas hier boost aan als je RANDOM FOREST GEBRUIKT!!
for i in range(len(clf_predictions)):
    pred_1.append(clf_predictions[i][1])

test_results['Probability_of_booking'] = pred_1


In [22]:
test_results

,srch_id,prop_id,Probability_of_booking
0,1,3180,0.05
1,1,5543,0.09
2,1,14142,0.05
3,1,22393,0.02
4,1,24194,0.07
...,...,...,...
4959178,332787,32019,0.00
4959179,332787,33959,0.19
4959180,332787,35240,0.00
4959181,332787,94437,0.02


In [23]:

final_results = test_results.sort_values(by = ['srch_id','Probability_of_booking'], ascending=[True,False])
final_results = final_results.drop('Probability_of_booking', 1)

# PAS DIRECTORY AAN NAAR GOEDE WAAR DIE HEEN MOET!
final_results.to_csv('random_forest_predict.csv', index=False)

In [24]:
final_results

,srch_id,prop_id
6,1,34263
16,1,74045
9,1,54937
21,1,94729
12,1,61934
...,...,...
4959181,332787,94437
4959176,332787,22854
4959177,332787,29018
4959178,332787,32019
